In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats

In [2]:
#get all of positions from vcf files
def getAllPositions(filepath):
    pos=[]
    vcf_file = pysam.VariantFile(filepath)
    for record in vcf_file:
        pos.append(record.pos)
    pos=sorted(list(set(pos)))
    return pos

In [3]:
def findPosition(filepath,start,length):
    arr=[]
    allpos=getAllPositions(filepath=filepath)
    arr.append(allpos[start*1000])
    if (start+length)*1000-1<len(allpos):
        end=allpos[(start+length)*1000-1]
    else:
        end=allpos[-1]
    arr.append(end)
    return arr



In [4]:
#Get the real position of corresponding to consecutive '1' in split_chr...
def GetPosition(filepath,res,threshold):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]
    filename=(filepath.split("/")[-1]).split("_start_lenth.csv")[0]
    findfoldername="split_"+filename.split("_")[0]
    findfoldername_filename=filename.split("_")[1]
    filepath="../"+findfoldername+"/"+findfoldername_filename+".vcf.gz"
    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue
    if midarr!=[]:
        res[findfoldername+"_"+findfoldername_filename]=midarr
    return res

In [5]:
#Get the real position of corresponding to consecutive '1' in mhc region
def getPositionforMHC(res):
    mhc_start_length_path="../csvfile_start_lengthwithoutX/mhc_start_lenth.csv"
    csv_data = pd.read_csv(mhc_start_length_path, low_memory = False)#防止弹出警告
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]
    for k,v in mydic.items():
        if v>=6:
            midarr.append(findPosition(filepath="../mhcdataset/1kgp.29720000-33130000.vcf",start=k,length=v))            
        else:
            continue
    res["MHC"]=midarr
    return res
            



In [6]:
res=dict()
mhcres=dict()
for curDir, dirs, files in os.walk(top="../csvfile_start_lengthwithoutX/"):
    for file in files:
            filename=file[:file.index(".")]
            if filename!="mhc_start_lenth":
                filepath=os.path.join(curDir,file)
                GetPosition(filepath=filepath,res=res,threshold=6)
            else:
                getPositionforMHC(res=mhcres)

filepath ../split_chr14/xaa.vcf.gz


[E::idx_find_and_load] Could not retrieve index file for '../split_chr14/xaa.vcf.gz'


filepath ../split_chr7/xbc.vcf.gz


[E::idx_find_and_load] Could not retrieve index file for '../split_chr7/xbc.vcf.gz'


filepath ../split_chr1/xak.vcf.gz


[E::idx_find_and_load] Could not retrieve index file for '../split_chr1/xak.vcf.gz'


filepath ../split_chr1/xbk.vcf.gz


[E::idx_find_and_load] Could not retrieve index file for '../split_chr1/xbk.vcf.gz'


In [7]:
res#complementary region for split_chr...

{'split_chr14_xaa': [[19806333, 19976578]],
 'split_chr7_xbc': [[124810550, 125070920]],
 'split_chr1_xak': [[45762038, 46100296]],
 'split_chr1_xbk': [[188243362, 188513984]]}

In [8]:
mhcres#complementary region for mhc region

{'MHC': [[32453110, 32577355], [32589647, 32716541]]}